# Testing out the Realsense Video Camera

## This cell adapted from a realsense example just displays aligned RGB and depth

In [1]:

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Start streaming
pipeline.start(config)

try:
    while True:

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        

        #Show image somehow
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)


finally:

    # Stop streaming
    pipeline.stop()


KeyboardInterrupt: 

In [4]:
!which pip

/home/svl/.local/bin/pip


# Test Video Capture (code from online)

In [22]:
def encode_depth(frame):
    frame = ((frame/12000)*255).astype(np.uint8)
    return np.repeat(frame[:, :, np.newaxis], 3, axis=-1)

In [27]:
import cv2
import numpy as np
import pyrealsense2 as rs
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = 640
frame_height = 480
num_frames = 100
fps = 5
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out_color = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width,frame_height))
out_depth = cv2.VideoWriter('depth.avi',cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width,frame_height))
    
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, 30)
config.enable_record_to_file('test.bag')

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Start streaming
pipeline.start(config)

try:
    for i in range(num_frames):

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        

        #Show image somehow
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))

        #Write RGB to video
        out_color.write(color_image)
        out_depth.write(encode_depth(depth_image))

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1000 // fps)


finally:

    # Stop streaming
    pipeline.stop()
    out_color.release()
    out_depth.release()


In [26]:
encode_depth(depth_image).min()

0

In [12]:
np.max(color_image)

255

In [9]:
np.expand_dims(depth_image, -1).shape

(480, 640, 1)

In [10]:
np.append(color_image, np.expand_dims(depth_image, -1), axis = -1).shape

(480, 640, 4)

# Full data upload code

In [1]:
def encode_depth(frame):
    frame = ((frame/12000)*255).astype(np.uint8)
    return np.repeat(frame[:, :, np.newaxis], 3, axis=-1)

In [8]:
def init_vids(filename, fps, frame_width, frame_height):
    out_color = cv2.VideoWriter(filename + '_color.mp4',cv2.VideoWriter_fourcc(*'hvc1'), fps, (frame_width,frame_height))
    out_depth = cv2.VideoWriter(filename + '_depth.mp4',cv2.VideoWriter_fourcc(*'hvc1'), fps, (frame_width,frame_height))
    return out_color, out_depth

In [42]:
import os
import pysftp as sftp
from multiprocessing import Process
from threading import Thread
import time

def sftpPut(filename, vid, host = 'aicare.stanford.edu'):
    vid.release()
    
    while True:
        try:
            with sftp.Connection(host= host, username ='danjwu', private_key='/home/svl/.ssh/id_rsa') as s:
                remotepath = os.path.join('/home/danjwu/data/', filename)
                localpath = os.path.join('/home/svl/Desktop/RealSense_Data_Ingestion/', filename)
                s.put(localpath, remotepath)
                print('Uploaded', filename)
                return
        except:
            #Try every 5 secs till it works
            time.sleep(5)
            
def dispatchPut(filename, vid):

    x = Thread(target=sftpPut, args=(filename, vid))
    x.start()


In [18]:
def tic():
    #Homemade version of matlab tic and toc functions
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc(name):
    if 'startTime_for_tictoc' in globals():
        print(name + ": Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")

In [46]:
import cv2
import numpy as np
import pyrealsense2 as rs
 
# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = 640
frame_height = 480
num_frames = 300
fps = 5
    
target_frame_time = 1/fps
    
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, frame_width, frame_height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, frame_width, frame_height, rs.format.bgr8, 30)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Start streaming
pipeline.start(config)

for j in range(10):
    tic()
    out_color, out_depth = init_vids(str(j), fps, frame_width, frame_height)
    toc("Video Init " + str(j))
    
    try:
        for i in range(num_frames):
            
            tic()
            
            frame_start_time = time.time()
            
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            # Align the depth frame to color frame
            aligned_frames = align.process(frames)

            # Get aligned frames
            depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())


            #Show image somehow
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

            # Stack both images horizontally
            #images = np.hstack((color_image, depth_colormap))

            #Write RGB to video
            out_color.write(color_image)
            out_depth.write(encode_depth(depth_image))

            # Show images
            #cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            #cv2.imshow('RealSense', images)
            curr_frame_time = time.time() - frame_start_time
            if curr_frame_time < target_frame_time:
                time.sleep(target_frame_time - curr_frame_time)
            
            toc("Frame " + str(i))


    finally:
        #tic()
        # Stop streaming
        #dispatchFinalize()
        #out_depth.release()
        #toc("Video " + str(j))
        tic()

        #Upload files
        dispatchPut(str(j) + '_color.mp4', out_color)
        dispatchPut(str(j) + '_depth.mp4', out_depth)
        toc("Video " + str(j))
pipeline.stop()

Video Init 0: Elapsed time is 0.049558401107788086 seconds.
Frame 0: Elapsed time is 0.20061182975769043 seconds.
Frame 1: Elapsed time is 0.2003626823425293 seconds.
Frame 2: Elapsed time is 0.20028114318847656 seconds.
Frame 3: Elapsed time is 0.20024466514587402 seconds.
Frame 4: Elapsed time is 0.20032024383544922 seconds.
Frame 5: Elapsed time is 0.20041275024414062 seconds.
Frame 6: Elapsed time is 0.20026087760925293 seconds.
Frame 7: Elapsed time is 0.20067572593688965 seconds.
Frame 8: Elapsed time is 0.20052337646484375 seconds.
Frame 9: Elapsed time is 0.20055484771728516 seconds.
Frame 10: Elapsed time is 0.200239896774292 seconds.
Frame 11: Elapsed time is 0.20034098625183105 seconds.
Frame 12: Elapsed time is 0.2006230354309082 seconds.
Frame 13: Elapsed time is 0.20023131370544434 seconds.
Frame 14: Elapsed time is 0.2003178596496582 seconds.
Frame 15: Elapsed time is 0.20067739486694336 seconds.
Frame 16: Elapsed time is 0.2021176815032959 seconds.
Frame 17: Elapsed tim